# Fractal Dimension Exploration

This notebook provides a deep dive into fractal dimension analysis in MyceliumFractalNet:
- Box-counting algorithm
- Fractal dimension computation
- Self-similarity analysis
- Pattern complexity characterization

In [ ]:
# Install required packages if running in Colab
import sys

if "google.colab" in sys.modules:
    !pip install -q mycelium-fractal-net matplotlib numpy scipy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

from mycelium_fractal_net import (
    estimate_fractal_dimension,
    make_simulation_config_demo,
    run_mycelium_simulation_with_history,
)

print("✓ Imports successful")

## 1. Understanding Fractal Dimension

Fractal dimension is a measure of pattern complexity. The box-counting dimension is computed as:

$$D = \lim_{\epsilon \to 0} \frac{\ln N(\epsilon)}{\ln(1/\epsilon)}$$

where $N(\epsilon)$ is the number of boxes of size $\epsilon$ needed to cover the pattern.

In [ ]:
# Generate a simulation
config = make_simulation_config_demo()
result = run_mycelium_simulation_with_history(config)

# Threshold field to create binary pattern
threshold = -60  # mV
binary_field = result.field_final > threshold

print(f"Field shape: {result.field_final.shape}")
print(f"Threshold: {threshold} mV")
print(f"Active pixels: {binary_field.sum()} ({binary_field.sum() / binary_field.size * 100:.1f}%)")

# Compute fractal dimension
D = estimate_fractal_dimension(binary_field)
print(f"\nFractal Dimension: {D:.4f}")
print("  1D line: D ≈ 1.0")
print("  2D plane: D ≈ 2.0")
print("  Complex pattern: 1.0 < D < 2.0")

## 2. Visualize Binary Pattern

Show the thresholded field used for fractal analysis.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Original field
im1 = axes[0].imshow(result.field_final, cmap="viridis", vmin=-100, vmax=50)
axes[0].axhline(y=result.field_final.shape[0] // 2, color="red", linestyle="--", alpha=0.3)
axes[0].set_title("Original Field")
axes[0].set_xlabel("X")
axes[0].set_ylabel("Y")
plt.colorbar(im1, ax=axes[0], label="Potential (mV)")

# Threshold visualization
axes[1].hist(result.field_final.flatten(), bins=50, alpha=0.7, edgecolor="black")
axes[1].axvline(
    threshold, color="red", linestyle="--", linewidth=2, label=f"Threshold: {threshold} mV"
)
axes[1].set_xlabel("Membrane Potential (mV)")
axes[1].set_ylabel("Frequency")
axes[1].set_title("Potential Distribution")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Binary pattern
axes[2].imshow(binary_field, cmap="binary", interpolation="nearest")
axes[2].set_title(f"Binary Pattern (D = {D:.3f})")
axes[2].set_xlabel("X")
axes[2].set_ylabel("Y")

plt.tight_layout()
plt.show()

## 3. Box-Counting Algorithm Visualization

Visualize how the box-counting algorithm works at different scales.

In [ ]:
def count_boxes_at_scale(binary_field, box_size):
    """
    Count non-empty boxes at a given scale.

    Returns:
        count: Number of non-empty boxes
        box_positions: List of (row, col) positions of non-empty boxes
    """
    h, w = binary_field.shape
    count = 0
    positions = []

    for i in range(0, h, box_size):
        for j in range(0, w, box_size):
            box = binary_field[i : min(i + box_size, h), j : min(j + box_size, w)]
            if box.any():  # Box contains at least one active pixel
                count += 1
                positions.append((i, j))

    return count, positions


# Visualize at different scales
scales = [16, 8, 4, 2]
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, box_size in enumerate(scales):
    count, positions = count_boxes_at_scale(binary_field, box_size)

    # Plot binary field
    axes[idx].imshow(binary_field, cmap="binary", interpolation="nearest")

    # Draw boxes
    for i, j in positions:
        rect = Rectangle(
            (j - 0.5, i - 0.5),
            box_size,
            box_size,
            linewidth=1,
            edgecolor="red",
            facecolor="none",
            alpha=0.3,
        )
        axes[idx].add_patch(rect)

    axes[idx].set_title(f"Box size: {box_size}×{box_size}\nCount: {count} boxes")
    axes[idx].set_xlabel("X")
    axes[idx].set_ylabel("Y")

plt.suptitle("Box-Counting at Different Scales", fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

print("Red boxes indicate non-empty regions at each scale.")
print("Smaller boxes capture finer details of the pattern.")

## 4. Log-Log Plot for Fractal Dimension

The fractal dimension is the slope of the log-log plot of box count vs. box size.

In [ ]:
# Compute box counts at multiple scales
box_sizes = [2**i for i in range(1, 6)]  # [2, 4, 8, 16, 32]
counts = []

for size in box_sizes:
    count, _ = count_boxes_at_scale(binary_field, size)
    counts.append(count)

# Convert to log scale
log_sizes = np.log(1.0 / np.array(box_sizes))
log_counts = np.log(counts)

# Fit line
coeffs = np.polyfit(log_sizes, log_counts, 1)
slope = coeffs[0]
intercept = coeffs[1]
fit_line = slope * log_sizes + intercept

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(log_sizes, log_counts, s=100, alpha=0.7, edgecolor="black", label="Data points")
plt.plot(log_sizes, fit_line, "r--", linewidth=2, label=f"Fit: D = {slope:.3f}")
plt.xlabel("ln(1/ε)", fontsize=12)
plt.ylabel("ln(N(ε))", fontsize=12)
plt.title("Log-Log Plot for Fractal Dimension Estimation", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Add annotations
for i, (x, y, size) in enumerate(zip(log_sizes, log_counts, box_sizes)):
    plt.annotate(
        f"{size}×{size}",
        (x, y),
        textcoords="offset points",
        xytext=(10, -10),
        ha="left",
        fontsize=9,
        alpha=0.7,
    )

plt.tight_layout()
plt.show()

print(f"\nEstimated Fractal Dimension from log-log plot: D = {slope:.4f}")
print(f"Comparison with estimate_fractal_dimension(): D = {D:.4f}")
r2 = 1 - np.sum((log_counts - fit_line) ** 2) / np.sum((log_counts - log_counts.mean()) ** 2)
print(f"R² (fit quality): {r2:.4f}")

## 5. Effect of Threshold on Fractal Dimension

Explore how the choice of threshold affects the measured fractal dimension.

In [ ]:
# Test multiple thresholds
thresholds = np.linspace(-80, -40, 20)
dimensions = []
active_fractions = []

for thresh in thresholds:
    binary = result.field_final > thresh
    if binary.sum() > 10:  # Need some active pixels
        D_temp = estimate_fractal_dimension(binary)
        dimensions.append(D_temp)
        active_fractions.append(binary.sum() / binary.size)
    else:
        dimensions.append(np.nan)
        active_fractions.append(0)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Fractal dimension vs threshold
axes[0].plot(thresholds, dimensions, marker="o", linewidth=2, markersize=6)
axes[0].axvline(-60, color="red", linestyle="--", label="Default threshold (-60 mV)")
axes[0].set_xlabel("Threshold (mV)")
axes[0].set_ylabel("Fractal Dimension")
axes[0].set_title("Fractal Dimension vs. Threshold")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Active fraction vs threshold
axes[1].plot(thresholds, active_fractions, marker="s", linewidth=2, markersize=6, color="green")
axes[1].axvline(-60, color="red", linestyle="--", label="Default threshold (-60 mV)")
axes[1].set_xlabel("Threshold (mV)")
axes[1].set_ylabel("Active Fraction")
axes[1].set_title("Active Pixel Fraction vs. Threshold")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  - Lower thresholds → More active pixels → Higher dimension")
print("  - Higher thresholds → Fewer active pixels → Lower dimension")
print("  - Optimal threshold balances pattern preservation and noise reduction")

## 6. Compare Different Patterns

Generate patterns with different parameters and compare their fractal dimensions.

In [ ]:
# Generate patterns with different settings
configs = [
    {"alpha": 0.2, "turing_enabled": True, "label": "Low diffusion + Turing"},
    {"alpha": 0.5, "turing_enabled": True, "label": "High diffusion + Turing"},
    {"alpha": 0.2, "turing_enabled": False, "label": "Low diffusion, no Turing"},
    {"alpha": 0.5, "turing_enabled": False, "label": "High diffusion, no Turing"},
]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, cfg in enumerate(configs):
    # Run simulation
    config_temp = make_simulation_config_demo()
    config_temp.alpha = cfg["alpha"]
    config_temp.turing_enabled = cfg["turing_enabled"]
    config_temp.steps = 50
    result_temp = run_mycelium_simulation_with_history(config_temp)

    # Compute dimension
    binary_temp = result_temp.field_final > -60
    D_temp = estimate_fractal_dimension(binary_temp)

    # Plot field
    im = axes[0, idx].imshow(result_temp.field_final, cmap="viridis", vmin=-100, vmax=50)
    axes[0, idx].set_title(f"{cfg['label']}\nField")
    axes[0, idx].set_xlabel("X")
    if idx == 0:
        axes[0, idx].set_ylabel("Y")

    # Plot binary
    axes[1, idx].imshow(binary_temp, cmap="binary", interpolation="nearest")
    axes[1, idx].set_title(f"D = {D_temp:.3f}")
    axes[1, idx].set_xlabel("X")
    if idx == 0:
        axes[1, idx].set_ylabel("Y")

plt.suptitle("Fractal Dimensions for Different Simulation Parameters", fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

print("\nKey insights:")
print("  - Turing morphogenesis increases pattern complexity (higher D)")
print("  - Lower diffusion preserves local structure (higher D)")
print("  - High diffusion smooths patterns (lower D)")

## 7. Temporal Evolution of Fractal Dimension

Track how fractal dimension changes during simulation.

In [ ]:
# Compute dimension at multiple time points
time_points = range(0, len(result.field_history), 10)
temporal_dimensions = []

for t in time_points:
    binary_t = result.field_history[t] > -60
    if binary_t.sum() > 10:
        D_t = estimate_fractal_dimension(binary_t)
        temporal_dimensions.append(D_t)
    else:
        temporal_dimensions.append(np.nan)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(list(time_points), temporal_dimensions, marker="o", linewidth=2, markersize=6)
plt.xlabel("Time Step", fontsize=12)
plt.ylabel("Fractal Dimension", fontsize=12)
plt.title("Temporal Evolution of Fractal Dimension", fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=1.5, color="gray", linestyle=":", alpha=0.5, label="D = 1.5")
plt.legend()
plt.tight_layout()
plt.show()

print("\nFractal dimension evolution:")
print(f"  Initial (t=0): D = {temporal_dimensions[0]:.3f}")
print(f"  Final (t={list(time_points)[-1]}): D = {temporal_dimensions[-1]:.3f}")
print(f"  Change: ΔD = {temporal_dimensions[-1] - temporal_dimensions[0]:.3f}")

## Summary

This notebook demonstrated:
- ✓ Box-counting algorithm for fractal dimension
- ✓ Visualization of multi-scale box coverage
- ✓ Log-log plot analysis
- ✓ Effect of threshold selection
- ✓ Comparison across simulation parameters
- ✓ Temporal evolution of fractal dimension

Key takeaways:
- Fractal dimension quantifies pattern complexity (1.0 < D < 2.0)
- Turing morphogenesis creates more complex patterns
- Diffusion rate affects pattern structure
- Threshold selection impacts measured dimension

For more:
- `01_field_simulation.ipynb` - Field simulation basics
- `02_feature_analysis.ipynb` - Complete feature extraction